In [1]:
import sys
sys.version

'3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56) \n[GCC 7.3.0]'

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os


conf = SparkConf().setMaster("local").setAppName("LHS")
sc = SparkContext(conf=conf).getOrCreate()

# create hash function paramater
## 創造hash function的a,b和prime，並且將他們儲存成一個list

In [3]:
prime = """
33599 33601 33613 33617 33619 33623 33629 33637 33641 33647 
33679 33703 33713 33721 33739 33749 33751 33757 33767 33769 
33773 33791 33797 33809 33811 33827 33829 33851 33857 33863 
33871 33889 33893 33911 33923 33931 33937 33941 33961 33967 
33997 34019 34031 34033 34039 34057 34061 34123 34127 34129 
34141 34147 34157 34159 34171 34183 34211 34213 34217 34231 
34253 34259 34261 34267 34273 34283 34297 34301 34303 34313 
34319 34327 34337 34351 34361 34367 34369 34381 34403 34421 
34429 34439 34457 34469 34471 34483 34487 34499 34501 34511 
34513 34519 34537 34543 34549 34583 34589 34591 34603 34607 
34613 34631 34649 34651 34667 34673 34679 34687 34693 34703 
34721 34729 34739 34747 34757 34759 34763 34781 34807 34819 
34841 34843 34847 34849 34871 34877 34883 34897 34913 34919 
34939 34949 34961 34963 34981 35023 35027 35051 35053 35059 
35069 35081 35083 35089 35099 35107 35111 35117 35129 35141 
35149 35153 35159 35171 35201 35221 35227 35251 35257 35267 
35279 35281 35291 35311 35317 35323 35327 35339 35353 35363 
35381 35393 35401 35407 35419 35423 35437 35447 35449 35461 
35491 35507 35509 35521 35527 35531 35533 35537 35543 35569 
35573 35591 35593 35597 35603 35617 35671 35677 35729 35731 

"""
prime_list= list(map(int , prime.split()))

In [4]:
import random

hash_num = 100
a_para = list()
b_para = list()
prime = list()


for i in range(hash_num):
    a_para.append(random.randint(1, 100000))
    b_para.append(random.randint(1, 100000))

while(len(prime) < 100):
    temp = random.choice(prime_list)
    if temp not in prime:
        prime.append(temp)
        
hash_fun = [ele for ele in zip(a_para, b_para, prime)]

# Find singles in documents
### 從文章中找出所有shingle
### all_shingle儲存documents當中出現的所有shingle
### shingle_in_each_doc替每個文章儲存他們的shingle
### 並且mapping成一個數字

In [5]:
dir_path = 'athletics'
files = sorted(os.listdir(dir_path))
docs = list()
doc_counter = len(files)

temp_count = 0

for file in files:
    doc = ''
    with open(os.path.join(dir_path, file)) as fp:
         data = fp.readlines()
    for i in range(len(data)):
        doc += data[i].replace('\n', '').replace(',', '').replace('"', '').replace('.', '').lower()
    docs.append(doc.split())

In [6]:
all_shingle = list()
shingle_in_each_doc = list()
shingle_num = 0

for i in range(len(docs)):
    shingles_in_doc = set()
    for j in range(1, len(docs[i])-1):
        shingle = docs[i][j-1] + " " + docs[i][j] + " " + docs[i][j+1]
        shingles_in_doc.add(shingle)
        if shingle not in all_shingle:
            all_shingle.append((shingle, shingle_num))
            shingle_num += 1
    shingle_in_each_doc.append((i, shingles_in_doc))

### create shingles mapping

In [7]:
shingle_map = sc.parallelize(all_shingle)
reverse_shingle_map = shingle_map.map(lambda s: (s[1], s[0]))

In [8]:
# 這個或許可以改成RDD的方式
shingle_map = dict(all_shingle)
shingle_in_each_doc_map_res = []

for ele in shingle_in_each_doc:
    temp = []
    doc_num, shingles = ele
    temp.append(doc_num)
    temp2 = []
    for shingle in list(shingles):
        temp2.append(shingle_map[shingle])
    temp.append(temp2)
    shingle_in_each_doc_map_res.append(temp)

# min-hashing
### 將文章中每個shingle進行min hashing的計算
### res原本是(docID, (hashID, hash_res))的形式
### 後來將res改成signature得形式((hashID,docID), hash_res)
### 最後利用groupByKey與min取得signature

In [9]:
# test version
N = len(all_shingle)
def min_hash(s):
    res = []
    for i in range(len(hash_fun)):
        a, b, p = hash_fun[i]
        for j in range(len(s)):
            res.append((i,(a*s[j]+b)%p%N))
    return res

shingle_in_each_doc_RDD = sc.parallelize(shingle_in_each_doc_map_res)
min_hash_res = shingle_in_each_doc_RDD.mapValues(min_hash).flatMapValues(lambda s:s).map(lambda s: ((s[1][0], s[0]), s[1][1]))
signature = min_hash_res.groupByKey().map(lambda s:(s[0], min(s[1])))

# LSH
### 將rows轉換成band idx
### 以(bandID, bucketID)為key，col為value
### 將上述結果group起來，最後的value就是condidate pair

In [10]:
# paramater setting
band = 50
r = len(hash_fun) / band
bucket_size = 10000000

In [11]:
LSH_res = signature.map(lambda s: ((s[0][0]//r, s[0][1]), s[1])).map(lambda s: ((s[0][0], s[1]%bucket_size), s[0][1]))
candidate_pair = [ sorted(list(set(ele[1]))) for ele in LSH_res.groupByKey().filter(lambda s: len(s[1])>1).collect()]

In [12]:
def Jaccard_sim(doc1, doc2):
    set1 = shingle_in_each_doc[doc1][1]
    set2 = shingle_in_each_doc[doc2][1]
    inter = set1.intersection(set2)
    union = set1.union(set2)
    return len(inter)/len(union)

# 針對每一個candidate_pair計算他們的Jaccard_sim

In [13]:
Jaccard_res = dict()

candidate_pair = list(candidate_pair)

for ele in candidate_pair:
    for i in range(len(ele)):
        for j in range(i+1, len(ele)):
            if (ele[i], ele[j]) not in Jaccard_res.keys():
                res = Jaccard_sim(ele[i], ele[j])
                Jaccard_res[ele[i], ele[j]] = res

In [14]:
res = sorted(Jaccard_res.items(), key=lambda s:s[1], reverse=True)
res[:10]

[((11, 19), 1.0),
 ((51, 83), 1.0),
 ((46, 48), 0.7551020408163265),
 ((29, 34), 0.7069306930693069),
 ((48, 87), 0.5081585081585082),
 ((47, 48), 0.48300117233294254),
 ((22, 37), 0.480719794344473),
 ((13, 39), 0.3968253968253968),
 ((46, 87), 0.38515901060070673),
 ((46, 47), 0.36557231588287487)]

# 儲存檔案

In [15]:
with open("Outputfile.txt", 'w') as fp:
    for i in range(10):
        fp.write("({0},{1}):{2}\n".format(res[i][0][0]+1, res[i][0][1]+1, res[i][1]))